In [ ]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
struct_dir = '/home/eva/Documents/designs/p-IL-dimer/simulation-input/'
pdb_file = struct_dir+'p-IL_fromtleap.pdb'
prmtop_file = struct_dir+'p-IL.prmtop'
inpcrd_file = struct_dir+'p-IL.inpcrd'
work_dir = '/home/eva/Documents/gbsa-sims/cv-sims/metadynamics/p-IL-dimer/year2/pIL-6/'

In [ ]:
#struct_dir = '/home/eva/Documents/structures/structures-obc2-ff96/apCC-Di/'
#pdb_file = struct_dir+'apCC-Di_fromtleap.pdb'
#prmtop_file = struct_dir+'apCC-Di.prmtop'
#inpcrd_file = struct_dir+'apCC-Di.inpcrd'
#work_dir = '/home/eva/Documents/gbsa-sims/cv-sims/metadynamics/apCC-Di/apCC-Di-1/'

In [ ]:
system_code = 'p-IL dimer'

In [ ]:
t = np.arange(0.0, 1000.0, 0.004)

In [ ]:
traj = md.load(work_dir+'output.dcd', top = prmtop_file)
traj

In [ ]:
traj_2 = md.load(work_dir+'replicate-2/output.dcd', top = prmtop_file)
traj_2

In [ ]:
traj_3 = md.load(work_dir+'replicate-3/output.dcd', top = prmtop_file)
traj_3

In [ ]:
prot = md.load_pdb(pdb_file)

chain1 = [atom.index for atom in prot.topology.chain(0).atoms] #chain1
chain2 = [atom.index for atom in prot.topology.chain(1).atoms] #chain2
assembly = [atom.index for atom in prot.topology.atoms] #full assembly

chain1_top = md.load_pdb(pdb_file, atom_indices=chain1)
chain2_top = md.load_pdb(pdb_file, atom_indices=chain2)

In [ ]:
chain1_res = [res.name for res in chain1_top.topology.residues]

chain2_res = [res.name for res in chain2_top.topology.residues]

In [ ]:
if 'ACE' in chain1_res: # exlude caps from end-to-end calculation
    nter_1_res_idx = 1
    chain1_length = len(chain1_res)
    first_ca_chain1 = [atom.index for atom in prot.topology.chain(0).atoms if ((atom.residue.index == nter_1_res_idx) and (atom.name == 'CA'))]
    last_ca_chain1 = [atom.index for atom in prot.topology.chain(0).atoms if ((atom.residue.index == chain1_length-2) and (atom.name == 'CA'))]

else:
    nter_1_res_idx = 0
    chain1_length = len(chain1_res)
    first_ca_chain1 = [atom.index for atom in prot.topology.chain(0).atoms if ((atom.residue.index == nter_1_res_idx) and (atom.name == 'CA'))]
    last_ca_chain1 = [atom.index for atom in prot.topology.chain(0).atoms if ((atom.residue.index == chain1_length-1) and (atom.name == 'CA'))]

if 'ACE' in chain2_res:
    nter_2_res_idx = 1
    chain2_length = len(chain2_res)
    first_ca_chain2 = [atom.index for atom in prot.topology.chain(1).atoms if ((atom.residue.index == chain1_length+1) and (atom.name == 'CA'))]
    last_ca_chain2 = [atom.index for atom in prot.topology.chain(1).atoms if ((atom.residue.index == 2*chain1_length-2) and (atom.name == 'CA'))]
else:
    nter_2_res_idx = 0
    chain2_length = len(chain2_res)
    first_ca_chain2 = [atom.index for atom in prot.topology.chain(1).atoms if ((atom.residue.index == chain1_length) and (atom.name == 'CA'))]
    last_ca_chain2 = [atom.index for atom in prot.topology.chain(1).atoms if ((atom.residue.index == 2*chain1_length-1) and (atom.name == 'CA'))]


In [ ]:
first_ca_array = np.array([[first_ca_chain1[0], first_ca_chain2[0]], [0,0]])
last_ca_array = np.array([[last_ca_chain1[0], last_ca_chain2[0]], [0,0]])

In [ ]:
first_ca_end2end_1 = md.compute_distances(traj, first_ca_array)
first_ca_end2end_2 = md.compute_distances(traj_2, first_ca_array)
first_ca_end2end_3 = md.compute_distances(traj_3, first_ca_array)

In [ ]:
first_ca_end2end_A_1 = []

for i in first_ca_end2end_1:
    first_ca_end2end_A_1.append(10*i[0])

first_ca_end2end_A_2 = []

for i in first_ca_end2end_2:
    first_ca_end2end_A_2.append(10*i[0])

first_ca_end2end_A_3 = []

for i in first_ca_end2end_3:
    first_ca_end2end_A_3.append(10*i[0])

In [ ]:
last_ca_end2end_1 = md.compute_distances(traj, last_ca_array)
last_ca_end2end_2 = md.compute_distances(traj_2, last_ca_array)
last_ca_end2end_3 = md.compute_distances(traj_3, last_ca_array)

In [ ]:
last_ca_end2end_A_1 = []

for i in last_ca_end2end_1:
    last_ca_end2end_A_1.append(10*i[0])

last_ca_end2end_A_2 = []

for i in last_ca_end2end_2:
    last_ca_end2end_A_2.append(10*i[0])

last_ca_end2end_A_3 = []

for i in last_ca_end2end_3:
    last_ca_end2end_A_3.append(10*i[0])

In [ ]:
fig,ax = plt.subplots(3, sharex=True, sharey=True, figsize=(8,8))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0].plot(t, first_ca_end2end_A_1, label = 'replicate 1', color='blue')
ax[0].set_title('replicate 1', fontsize=10)
plt.tight_layout
ax[1].plot(t, first_ca_end2end_A_2, label = 'replicate 2', color='steelblue')
ax[1].set_title('replicate 2', fontsize=10)
ax[2].plot(t, first_ca_end2end_A_3, label = 'replicate 3', color='cornflowerblue')
ax[2].set_title('replicate 3', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('N-termini CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'N-termini-dist.png')
plt.show()


In [ ]:
fig,ax = plt.subplots(3, sharex=True, sharey=True, figsize=(8,8))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0].plot(t, last_ca_end2end_A_1, label = 'replicate 1', color='firebrick')
ax[0].set_title('replicate 1', fontsize=10)
plt.tight_layout
ax[1].plot(t, last_ca_end2end_A_2, label = 'replicate 2', color='red')
ax[1].set_title('replicate 2', fontsize=10)
ax[2].plot(t, last_ca_end2end_A_3, label = 'replicate 3', color='tomato')
ax[2].set_title('replicate 3', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('C-termini CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'C-termini-dist.png')
plt.show()


In [ ]:
# also need to make N-C termini distance and C-N termini distance to see if they adopt opposite topologies

In [ ]:
first_ca1_last_ca2_array = np.array([[first_ca_chain1[0], last_ca_chain2[0]], [0,0]])
last_ca1_first_ca2_array = np.array([[last_ca_chain1[0], first_ca_chain2[0]], [0,0]])

In [ ]:
first_ca1_last_ca2_end2end_1 = md.compute_distances(traj, first_ca1_last_ca2_array)
first_ca1_last_ca2_end2end_2 = md.compute_distances(traj_2, first_ca1_last_ca2_array)
first_ca1_last_ca2_end2end_3 = md.compute_distances(traj_3, first_ca1_last_ca2_array)

In [ ]:
first_ca1_last_ca2_end2end_A_1 = []

for i in first_ca1_last_ca2_end2end_1:
    first_ca1_last_ca2_end2end_A_1.append(10*i[0])

first_ca1_last_ca2_end2end_A_2 = []

for i in first_ca1_last_ca2_end2end_2:
    first_ca1_last_ca2_end2end_A_2.append(10*i[0])

first_ca1_last_ca2_end2end_A_3 = []

for i in first_ca1_last_ca2_end2end_3:
    first_ca1_last_ca2_end2end_A_3.append(10*i[0])

In [ ]:
fig,ax = plt.subplots(3, sharex=True, sharey=True, figsize=(8,8))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0].plot(t, first_ca1_last_ca2_end2end_A_1, label = 'replicate 1', color='darkgreen')
ax[0].set_title('replicate 1', fontsize=10)
plt.tight_layout
ax[1].plot(t, first_ca1_last_ca2_end2end_A_2, label = 'replicate 2', color='seagreen')
ax[1].set_title('replicate 2', fontsize=10)
ax[2].plot(t, first_ca1_last_ca2_end2end_A_3, label = 'replicate 3', color='limegreen')
ax[2].set_title('replicate 3', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('N-C termini CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'NC-termini-dist.png')
plt.show()


In [ ]:
last_ca1_first_ca2_end2end_1 = md.compute_distances(traj, last_ca1_first_ca2_array)
last_ca1_first_ca2_end2end_2 = md.compute_distances(traj_2, last_ca1_first_ca2_array)
last_ca1_first_ca2_end2end_3 = md.compute_distances(traj_3, last_ca1_first_ca2_array)

In [ ]:
last_ca1_first_ca2_end2end_A_1 = []

for i in last_ca1_first_ca2_end2end_1:
    last_ca1_first_ca2_end2end_A_1.append(10*i[0])

last_ca1_first_ca2_end2end_A_2 = []

for i in last_ca1_first_ca2_end2end_2:
    last_ca1_first_ca2_end2end_A_2.append(10*i[0])

last_ca1_first_ca2_end2end_A_3 = []

for i in last_ca1_first_ca2_end2end_3:
    last_ca1_first_ca2_end2end_A_3.append(10*i[0])

In [ ]:
fig,ax = plt.subplots(3, sharex=True, sharey=True, figsize=(8,8))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0].plot(t, last_ca1_first_ca2_end2end_A_1, label = 'replicate 1', color='indigo')
ax[0].set_title('replicate 1', fontsize=10)
plt.tight_layout
ax[1].plot(t, last_ca1_first_ca2_end2end_A_2, label = 'replicate 2', color='darkviolet')
ax[1].set_title('replicate 2', fontsize=10)
ax[2].plot(t, last_ca1_first_ca2_end2end_A_3, label = 'replicate 3', color='violet')
ax[2].set_title('replicate 3', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('C-N termini CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'CN-termini-dist.png')
plt.show()

In [ ]:
fig,ax = plt.subplots(2,2, sharex=True, sharey=True, figsize=(11,11))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0,0].plot(t, first_ca_end2end_A_1, label = 'N-N', color='cornflowerblue')
ax[0,0].set_title('N-N', fontsize=10)
ax[0,1].plot(t, last_ca_end2end_A_1, label = 'C-C', color='firebrick')
ax[0,1].set_title('C-C', fontsize=10)
ax[1,0].plot(t, first_ca1_last_ca2_end2end_A_1, label = 'N-C', color='seagreen')
ax[1,0].set_title('N-C', fontsize=10)
ax[1,1].plot(t, last_ca1_first_ca2_end2end_A_1, label = 'C-N', color='darkviolet')
ax[1,1].set_title('C-N', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('terminal CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'rep1-termini-dist.png')
plt.show()


In [ ]:
fig,ax = plt.subplots(2,2, sharex=True, sharey=True, figsize=(11,11))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0,0].plot(t, first_ca_end2end_A_2, label = 'N-N', color='cornflowerblue')
ax[0,0].set_title('N-N', fontsize=10)
ax[0,1].plot(t, last_ca_end2end_A_2, label = 'C-C', color='firebrick')
ax[0,1].set_title('C-C', fontsize=10)
ax[1,0].plot(t, first_ca1_last_ca2_end2end_A_2, label = 'N-C', color='seagreen')
ax[1,0].set_title('N-C', fontsize=10)
ax[1,1].plot(t, last_ca1_first_ca2_end2end_A_2, label = 'C-N', color='darkviolet')
ax[1,1].set_title('C-N', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('terminal CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'rep2-termini-dist.png')
plt.show()


In [ ]:
fig,ax = plt.subplots(2,2, sharex=True, sharey=True, figsize=(11,11))
fig.tight_layout(h_pad=2)
fig.suptitle(system_code, fontsize=15,y=1)
ax[0,0].plot(t, first_ca_end2end_A_3, label = 'N-N', color='cornflowerblue')
ax[0,0].set_title('N-N', fontsize=10)
ax[0,1].plot(t, last_ca_end2end_A_3, label = 'C-C', color='firebrick')
ax[0,1].set_title('C-C', fontsize=10)
ax[1,0].plot(t, first_ca1_last_ca2_end2end_A_3, label = 'N-C', color='seagreen')
ax[1,0].set_title('N-C', fontsize=10)
ax[1,1].plot(t, last_ca1_first_ca2_end2end_A_3, label = 'C-N', color='darkviolet')
ax[1,1].set_title('C-N', fontsize=10)
fig.supxlabel('t (ns)')
fig.supylabel('terminal CA distance ($\AA$)')
fig.tight_layout()
fig.savefig(work_dir+'rep3-termini-dist.png')
plt.show()
